In [ ]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta

# --- CONFIGURATION ---
NUM_NORMAL = 5000
NUM_FRAUD = 100
OUTPUT_FILE = "amanpay_transactions_improved.csv"

# --- HELPER FUNCTIONS ---
def generate_normal_transaction():
    """Simulates realistic student spending in Malaysia"""
    categories = ['Food', 'Transport', 'Telco', 'Grocery', 'Entertainment', 'Education']

    # More realistic spending distribution
    category = random.choice(categories)

    # Different spending patterns per category
    if category == 'Food':
        amount = round(np.random.gamma(3, 5), 2)  # RM 5-30 typically
    elif category == 'Transport':
        amount = round(np.random.gamma(2, 3), 2)  # RM 3-15
    elif category == 'Education':
        amount = round(np.random.uniform(50, 200), 2)  # Books, materials
    else:
        amount = round(np.random.gamma(3, 7), 2)

    amount = max(2.0, min(amount, 500))  # Cap at RM 500 for normal

    # Realistic hours - students are active during day
    # Peak times: lunch (12-2), dinner (6-8), late night (10-12)
    hour_weights = [1, 1, 1, 1, 1, 2, 3, 5, 7, 8, 6, 8, 10, 9, 7, 6, 5, 7, 9, 8, 6, 5, 4, 3]
    hour = random.choices(range(24), weights=hour_weights, k=1)[0]

    # Local coordinates (Kampar, KL, Penang areas)
    locations = [
        (4.3352, 101.1529),  # Kampar
        (3.1390, 101.6869),  # KL
        (5.4164, 100.3327),  # Penang
    ]
    base_lat, base_lon = random.choice(locations)
    lat = round(base_lat + random.uniform(-0.1, 0.1), 4)
    lon = round(base_lon + random.uniform(-0.1, 0.1), 4)

    return {
        "amount": amount,
        "hour": hour,
        "category": category,
        "lat": lat,
        "lon": lon,
        "is_fraud": 0
    }

def generate_fraud_transaction():
    """Simulates various types of fraud with realistic patterns"""
    fraud_types = ['BIG_DRAIN', 'PHISHING', 'SMALL_REPEATED', 'OVERSEAS', 'ODD_HOURS']
    fraud_type = random.choice(fraud_types)

    if fraud_type == 'BIG_DRAIN':
        # Large unauthorized transfer
        amount = round(random.uniform(1500, 5000), 2)
        hour = random.randint(0, 6)  # Late night/early morning
        category = "Transfer"
        # Could be local or overseas
        if random.random() < 0.5:
            lat = round(random.uniform(3.0, 5.0), 4)
            lon = round(random.uniform(101.0, 102.0), 4)
        else:
            lat = round(random.uniform(10.0, 50.0), 4)
            lon = round(random.uniform(110.0, 130.0), 4)

    elif fraud_type == 'PHISHING':
        # Fake website/app - specific amounts
        amount = random.choice([199.00, 299.00, 99.00, 499.00])
        hour = random.randint(0, 23)  # Can happen anytime
        category = random.choice(["Transfer", "Entertainment", "Shopping"])
        # Overseas locations
        lat = round(random.uniform(15.0, 40.0), 4)
        lon = round(random.uniform(100.0, 125.0), 4)

    elif fraud_type == 'SMALL_REPEATED':
        # Testing stolen card with small amounts
        amount = round(random.uniform(5, 50), 2)
        hour = random.randint(0, 5)  # Odd hours
        category = random.choice(["Food", "Grocery", "Entertainment"])
        # Foreign location
        lat = round(random.uniform(20.0, 50.0), 4)
        lon = round(random.uniform(105.0, 130.0), 4)

    elif fraud_type == 'OVERSEAS':
        # Legitimate-looking amount but weird location
        amount = round(random.uniform(100, 800), 2)
        hour = random.randint(8, 23)  # Normal hours
        category = random.choice(["Shopping", "Entertainment", "Transfer"])
        # Definitely overseas (China, Nigeria, Eastern Europe)
        regions = [
            (35.0, 105.0, 45.0, 115.0),  # China
            (8.0, 3.0, 12.0, 8.0),       # Nigeria
            (45.0, 20.0, 55.0, 40.0),    # Eastern Europe
        ]
        region = random.choice(regions)
        lat = round(random.uniform(region[0], region[2]), 4)
        lon = round(random.uniform(region[1], region[3]), 4)

    else:  # ODD_HOURS
        # Normal amount but suspicious timing
        amount = round(random.uniform(200, 1000), 2)
        hour = random.randint(1, 4)  # 1-4 AM
        category = "Transfer"
        # Could be local
        lat = round(random.uniform(3.0, 5.5), 4)
        lon = round(random.uniform(100.5, 102.0), 4)

    return {
        "amount": amount,
        "hour": hour,
        "category": category,
        "lat": lat,
        "lon": lon,
        "is_fraud": 1
    }

# --- MAIN EXECUTION ---
print("🔄 Generating improved dataset with realistic patterns...")

data = []

# Generate normal transactions
for _ in range(NUM_NORMAL):
    data.append(generate_normal_transaction())

# Generate fraud transactions
for _ in range(NUM_FRAUD):
    data.append(generate_fraud_transaction())

# Convert to DataFrame
df = pd.DataFrame(data)

# Shuffle the data
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# Save to CSV
df.to_csv(OUTPUT_FILE, index=False)

# Show statistics
print(f"✅ Generated {len(df)} transactions")
print(f"   - Normal: {len(df[df['is_fraud']==0])} ({len(df[df['is_fraud']==0])/len(df)*100:.1f}%)")
print(f"   - Fraud: {len(df[df['is_fraud']==1])} ({len(df[df['is_fraud']==1])/len(df)*100:.1f}%)")
print(f"\n📊 Fraud Amount Range: RM {df[df['is_fraud']==1]['amount'].min():.2f} - RM {df[df['is_fraud']==1]['amount'].max():.2f}")
print(f"📊 Normal Amount Range: RM {df[df['is_fraud']==0]['amount'].min():.2f} - RM {df[df['is_fraud']==0]['amount'].max():.2f}")
print(f"\n💾 Saved to '{OUTPUT_FILE}'")

🔄 Generating improved dataset with realistic patterns...
✅ Generated 5100 transactions
   - Normal: 5000 (98.0%)
   - Fraud: 100 (2.0%)

📊 Fraud Amount Range: RM 6.75 - RM 4897.50
📊 Normal Amount Range: RM 2.00 - RM 199.87

💾 Saved to 'amanpay_transactions.csv'


In [ ]:
df

,amount,hour,category,lat,lon,is_fraud
0,10.09,8,Telco,5.3372,100.2795,0
1,32.76,12,Telco,5.4146,100.3291,0
2,10.39,13,Grocery,4.3639,101.1646,0
3,15.63,8,Telco,3.1023,101.7447,0
4,199.89,8,Education,4.2475,101.1211,0
...,...,...,...,...,...,...
5095,7.02,8,Food,3.1832,101.6926,0
5096,128.87,17,Education,5.3395,100.4033,0
5097,3.83,9,Transport,3.0875,101.6834,0
5098,5.41,12,Telco,4.3908,101.1444,0


In [ ]:
import pandas as pd

# --- CONFIGURATION ---
INPUT_FILE = "amanpay_transactions.csv"
OUTPUT_FILE = "amanpay_cleaned.csv"

def clean_data():
    print("🧹 Starting Data Cleaning Process...")

    # 1. Load the Data
    try:
        df = pd.read_csv(INPUT_FILE)
        print(f"   -> Loaded {len(df)} rows.")
    except FileNotFoundError:
        print("❌ Error: generate_data.py hasn't been run yet!")
        return

    # 2. Check for Missing Values (and remove them)
    if df.isnull().sum().sum() > 0:
        print(f"   -> Found missing values. Removing...")
        df = df.dropna()
    else:
        print("   -> No missing values found. (Good!)")

    # 3. Remove Duplicates
    # In real banking, duplicates might happen due to system glitches
    initial_count = len(df)
    df = df.drop_duplicates()
    final_count = len(df)

    if initial_count > final_count:
        print(f"   -> Removed {initial_count - final_count} duplicate rows.")
    else:
        print("   -> No duplicates found.")

    # 4. Data Type Conversion (Optimization)
    # Ensure 'amount' is a number (float)
    df['amount'] = pd.to_numeric(df['amount'], errors='coerce')

    # 5. Save the Cleaned Data
    df.to_csv(OUTPUT_FILE, index=False)
    print("------------------------------------------------")
    print(f"Data Cleaning Complete!")
    print(f"   -> Saved cleaned data to: '{OUTPUT_FILE}'")
    print("------------------------------------------------")

    # Show a quick summary for the User
    print("\nData Preview (First 5 rows):")
    print(df.head())
    print("\nStatistics:")
    print(df.describe())

if __name__ == "__main__":
    clean_data()

🧹 Starting Data Cleaning Process...
   -> Loaded 5100 rows.
   -> No missing values found. (Good!)
   -> No duplicates found.
------------------------------------------------
Data Cleaning Complete!
   -> Saved cleaned data to: 'amanpay_cleaned.csv'
------------------------------------------------

Data Preview (First 5 rows):
   amount  hour   category     lat       lon  is_fraud
0    5.79    10  Transport  3.1427  101.6293         0
1   14.77    14    Grocery  5.3434  100.3573         0
2   11.81     4    Grocery  4.4134  101.2468         0
3   12.77    19       Food  5.4425  100.3840         0
4    5.79    12       Food  3.2247  101.6697         0

Statistics:
            amount         hour          lat          lon     is_fraud
count  5100.000000  5100.000000  5100.000000  5100.000000  5100.000000
mean     49.864124    13.341176     4.650604   100.992859     0.019608
std     185.551196     5.318164     3.548747     4.739280     0.138662
min       2.000000     0.000000     3.000400

In [ ]:
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras import layers, losses

# --- CONFIGURATION ---
INPUT_FILE = "amanpay_cleaned.csv"
MODEL_FILE = "amanpay_model.keras"  # The saved "Brain"
SCALER_FILE = "scaler.pkl"          # The saved "Translator"

# 1. Load the Data
print("Loading data...")
df = pd.read_csv(INPUT_FILE)

# We only care about these columns for the AI
features = ['amount', 'hour', 'lat', 'lon']
data = df[features]

# 2. Preprocess (Scale the numbers)
# AI works better when numbers are small (between 0 and 1)
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data)

# 3. Split Data
# We train ONLY on Normal data (is_fraud = 0)
# This is called "Anomaly Detection"
normal_data = data_scaled[df['is_fraud'] == 0]
train_data, test_data = train_test_split(normal_data, test_size=0.2, random_state=42)

print(f"Training on {len(train_data)} normal transactions...")

# 4. Build the TensorFlow Model (Autoencoder)
# It tries to compress the data and then recreate it.
# If it fails to recreate a transaction, that transaction is a SCAM.
input_dim = data_scaled.shape[1] # 4 features

model = tf.keras.Sequential([
  # Encoder (Compress)
  layers.Dense(8, activation="relu", input_shape=(input_dim,)),
  layers.Dense(4, activation="relu"),

  # Decoder (Expand)
  layers.Dense(8, activation="relu"),
  layers.Dense(input_dim, activation="sigmoid")
])

model.compile(optimizer='adam', loss='mae')

# 5. Train the Model
history = model.fit(train_data, train_data,
                    epochs=20,
                    batch_size=32,
                    validation_data=(test_data, test_data),
                    shuffle=True,
                    verbose=1)

# 6. Save the Brain and Scaler
model.save(MODEL_FILE)
with open(SCALER_FILE, 'wb') as f:
    pickle.dump(scaler, f)

print("Success! Model trained and saved.")
print(f"Files created: {MODEL_FILE}, {SCALER_FILE}")

Loading data...
Training on 4000 normal transactions...
Epoch 1/20


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 0.6220 - val_loss: 0.5114
Epoch 2/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.4712 - val_loss: 0.3860
Epoch 3/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.3706 - val_loss: 0.3570
Epoch 4/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.3506 - val_loss: 0.3417
Epoch 5/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.3314 - val_loss: 0.3175
Epoch 6/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.3126 - val_loss: 0.2827
Epoch 7/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.2759 - val_loss: 0.2700
Epoch 8/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2675 - val_loss: 0.2672
Epoch 9/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2644 - val_loss: 0.2666
Epoch 10/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2637 - val_loss: 0.2657
Epoch 11/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2629 - val_loss: 0.2651
Epoch 12/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.

In [ ]:
pip install streamlit

In [ ]:
%%writefile app.py
import streamlit as st
import pandas as pd
import numpy as np
import pickle
import tensorflow as tf
import google.generativeai as genai
import os
import firebase_admin
from firebase_admin import credentials
from firebase_admin import db
import time

# --- 1. CONFIGURATION ---
MODEL_FILE = "amanpay_model.keras"
SCALER_FILE = "scaler.pkl"
FIREBASE_KEY = "key.json"  # Ensure you uploaded this file!

# PASTE YOUR GOOGLE GEMINI API KEY HERE
GOOG_API_KEY = "YOUR_API_KEY"

# --- 2. SETUP FIREBASE ---
# We use a "Try/Except" block to prevent errors when Streamlit reloads
if not firebase_admin._apps:
    try:
        cred = credentials.Certificate(FIREBASE_KEY)
        firebase_admin.initialize_app(cred, {
            'databaseURL': 'OWN_FIREBASE_LINK'
            # ^^^ IMPORTANT: REPLACE THIS URL WITH YOUR OWN FIREBASE URL IF IT FAILS ^^^
        })
        st.toast("🔥 Firebase Connected!", icon="☁️")
    except Exception as e:
        st.warning(f"Firebase not connected: {e}")

# --- 3. AUTO-DETECT GEMINI MODEL ---
gemini_active = False
model_gemini = None
try:
    if GOOG_API_KEY.startswith("AIza"):
        genai.configure(api_key=GOOG_API_KEY)
        available_models = [m.name for m in genai.list_models() if 'generateContent' in m.supported_generation_methods]
        chosen_model = next((m for m in available_models if "gemini" in m), available_models[0] if available_models else None)
        if chosen_model:
            model_gemini = genai.GenerativeModel(chosen_model)
            gemini_active = True
except:
    gemini_active = False

# --- 4. LOAD BRAIN ---
@st.cache_resource
def load_resources():
    if not os.path.exists(MODEL_FILE): return None, None
    model = tf.keras.models.load_model(MODEL_FILE)
    with open(SCALER_FILE, 'rb') as f: scaler = pickle.load(f)
    return model, scaler

try:
    model, scaler = load_resources()
    if model is None: st.stop()
except: st.stop()

# --- 5. UI LAYOUT ---
st.set_page_config(page_title="AmanPay AI", page_icon="🛡️", layout="wide")
st.title("🛡️ AmanPay AI")
st.markdown("### Securing the Digital Economy for Malaysia’s Underserved")
st.markdown("---")

# --- 6. SIDEBAR INPUTS ---
st.sidebar.header("📝 Transaction Simulator")
amount = st.sidebar.number_input("Amount (RM)", min_value=1.0, value=20.0, step=10.0)
hour = st.sidebar.slider("Time of Day (24h)", 0, 23, 14)
st.sidebar.subheader("📍 Location Details")
lat = st.sidebar.number_input("Latitude", value=3.1408, format="%.4f")
lon = st.sidebar.number_input("Longitude", value=101.6932, format="%.4f")
btn = st.sidebar.button("🚀 Analyze Transaction", type="primary")

# --- 7. MAIN LOGIC ---
if btn:
    # Scale & Predict
    input_data = pd.DataFrame([[amount, hour, lat, lon]], columns=['amount', 'hour', 'lat', 'lon'])
    input_scaled = scaler.transform(input_data)
    reconstructed = model.predict(input_scaled)
    error_score = np.mean(np.power(input_scaled - reconstructed, 2), axis=1)[0]
    is_fraud = error_score > 0.5

    # === FIREBASE LOGGING ===
    # This sends the data to the cloud instantly
    try:
        ref = db.reference("/transactions")
        ref.push({
            "amount": amount,
            "hour": hour,
            "risk_score": float(error_score),
            "status": "FRAUD" if is_fraud else "SAFE",
            "timestamp": time.time()
        })
        st.toast("Data synced to Cloud Database", icon="✅")
    except Exception as e:
        print(f"Firebase Error: {e}")

    # === DISPLAY RESULTS ===
    col1, col2 = st.columns([2, 1])
    with col1:
        st.subheader("📊 Analysis Result")
        if is_fraud:
            st.error(f"🚨 FRAUD DETECTED!")
            st.metric("Risk Score", f"{error_score:.4f}", "High Risk", delta_color="inverse")

            st.markdown("#### 🤖 AI Guardian Explanation:")
            if gemini_active:
                with st.spinner("Analyzing context..."):
                    try:
                        prompt = f"You're a friendly Malaysian security bot. Transaction: RM{amount} at {hour}:00, location {lat},{lon}. Risk score {error_score:.2f}. Explain in 1 casual sentence why this is blocked, using Malaysian slang like 'lah', 'lor', 'wei'. Be protective but friendly."
                        response = model_gemini.generate_content(prompt)
                        st.info(response.text)
                    except: st.error("AI Busy.")
        else:
            st.success(f"✅ Transaction Approved")
            st.metric("Risk Score", f"{error_score:.4f}", "Safe")

    with col2:
        st.subheader("🌍 Location Tracker")
        st.map(pd.DataFrame({'lat': [lat], 'lon': [lon]}))

# --- 8. LIVE HISTORY SECTION ---
st.markdown("---")
st.subheader("☁️ Live Cloud Database Records")
try:
    # Read the last 5 transactions from Firebase
    ref = db.reference("/transactions")
    snapshot = ref.order_by_key().limit_to_last(5).get()

    if snapshot:
        # Convert Firebase JSON to a nice Table
        data_list = []
        for key, val in snapshot.items():
            data_list.append(val)
        df_history = pd.DataFrame(data_list)
        # Reorder columns for neatness
        st.dataframe(df_history[['status', 'amount', 'risk_score', 'hour']], use_container_width=True)
    else:
        st.write("No records in cloud yet.")
except:
    st.write("Connect Firebase to see history.")

Overwriting app.py


In [ ]:
%%writefile app.py
import streamlit as st
import pandas as pd
import numpy as np
import pickle
import tensorflow as tf
import google.generativeai as genai
import os
import firebase_admin
from firebase_admin import credentials
from firebase_admin import db
import time

# --- 1. CONFIGURATION ---
MODEL_FILE = "amanpay_model.keras"
SCALER_FILE = "scaler.pkl"
FIREBASE_KEY = "key.json"  # Ensure you uploaded this file!

# !!! PASTE YOUR GOOGLE GEMINI API KEY HERE !!!
GOOG_API_KEY = "YOUR_API_KEY"

# --- 2. SETUP FIREBASE ---
# We use a "Try/Except" block to prevent errors when Streamlit reloads
if not firebase_admin._apps:
    try:
        cred = credentials.Certificate(FIREBASE_KEY)
        firebase_admin.initialize_app(cred, {
            'databaseURL': 'YOUR_DATABASE_LINK'
            # ^^^ IMPORTANT: REPLACE THIS URL WITH YOUR OWN FIREBASE URL IF IT FAILS ^^^
        })
        st.toast("🔥 Firebase Connected!", icon="☁️")
    except Exception as e:
        st.warning(f"Firebase not connected: {e}")

# --- 3. AUTO-DETECT GEMINI MODEL ---
gemini_active = False
model_gemini = None
try:
    if GOOG_API_KEY.startswith("AIza"):
        genai.configure(api_key=GOOG_API_KEY)
        available_models = [m.name for m in genai.list_models() if 'generateContent' in m.supported_generation_methods]
        chosen_model = next((m for m in available_models if "gemini" in m), available_models[0] if available_models else None)
        if chosen_model:
            model_gemini = genai.GenerativeModel(chosen_model)
            gemini_active = True
except:
    gemini_active = False

# --- 4. LOAD BRAIN ---
@st.cache_resource
def load_resources():
    if not os.path.exists(MODEL_FILE): return None, None
    model = tf.keras.models.load_model(MODEL_FILE)
    with open(SCALER_FILE, 'rb') as f: scaler = pickle.load(f)
    return model, scaler

try:
    model, scaler = load_resources()
    if model is None: st.stop()
except: st.stop()

# --- 5. UI LAYOUT ---
st.set_page_config(page_title="AmanPay AI", page_icon="🛡️", layout="wide")
st.title("🛡️ AmanPay AI")
st.markdown("### Securing the Digital Economy for Malaysia’s Underserved")
st.markdown("---")

# --- 6. SIDEBAR INPUTS ---
st.sidebar.header("📝 Transaction Simulator")
amount = st.sidebar.number_input("Amount (RM)", min_value=1.0, value=20.0, step=10.0)
hour = st.sidebar.slider("Time of Day (24h)", 0, 23, 14)
st.sidebar.subheader("📍 Location Details")
lat = st.sidebar.number_input("Latitude", value=3.1408, format="%.4f")
lon = st.sidebar.number_input("Longitude", value=101.6932, format="%.4f")
btn = st.sidebar.button("🚀 Analyze Transaction", type="primary")

# --- 7. MAIN LOGIC ---
if btn:
    # Scale & Predict
    input_data = pd.DataFrame([[amount, hour, lat, lon]], columns=['amount', 'hour', 'lat', 'lon'])
    input_scaled = scaler.transform(input_data)
    reconstructed = model.predict(input_scaled)
    error_score = np.mean(np.power(input_scaled - reconstructed, 2), axis=1)[0]
    is_fraud = error_score > 0.5

    # === FIREBASE LOGGING ===
    # This sends the data to the cloud instantly
    try:
        ref = db.reference("/transactions")
        ref.push({
            "amount": amount,
            "hour": hour,
            "risk_score": float(error_score),
            "status": "FRAUD" if is_fraud else "SAFE",
            "timestamp": time.time()
        })
        st.toast("Data synced to Cloud Database", icon="✅")
    except Exception as e:
        print(f"Firebase Error: {e}")

    # === DISPLAY RESULTS ===
    col1, col2 = st.columns([2, 1])
    with col1:
        st.subheader("📊 Analysis Result")
        if is_fraud:
            st.error(f"🚨 FRAUD DETECTED!")
            st.metric("Risk Score", f"{error_score:.4f}", "High Risk", delta_color="inverse")

            st.markdown("#### 🤖 AI Guardian Explanation:")
            if gemini_active:
                with st.spinner("Analyzing context..."):
                    try:
                        prompt = f"Act as a security bot. A user spent RM {amount} at hour {hour}. Location: {lat}, {lon}. Risk Score: {error_score:.2f}. Explain professionally in English why this is blocked in 1 sentence."
                        response = model_gemini.generate_content(prompt)
                        st.info(response.text)
                    except: st.error("AI Busy.")
        else:
            st.success(f"✅ Transaction Approved")
            st.metric("Risk Score", f"{error_score:.4f}", "Safe")

    with col2:
        st.subheader("🌍 Location Tracker")
        st.map(pd.DataFrame({'lat': [lat], 'lon': [lon]}))

# --- 8. LIVE HISTORY SECTION ---
st.markdown("---")
st.subheader("☁️ Live Cloud Database Records")
try:
    # Read the last 5 transactions from Firebase
    ref = db.reference("/transactions")
    snapshot = ref.order_by_key().limit_to_last(5).get()

    if snapshot:
        # Convert Firebase JSON to a nice Table
        data_list = []
        for key, val in snapshot.items():
            data_list.append(val)
        df_history = pd.DataFrame(data_list)
        # Reorder columns for neatness
        st.dataframe(df_history[['status', 'amount', 'risk_score', 'hour']], use_container_width=True)
    else:
        st.write("No records in cloud yet.")
except:
    st.write("Connect Firebase to see history.")

Overwriting app.py


In [ ]:
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared-linux-amd64
!mv cloudflared-linux-amd64 cloudflared

--2026-02-06 04:20:37--  https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/cloudflare/cloudflared/releases/download/2026.1.2/cloudflared-linux-amd64 [following]
--2026-02-06 04:20:37--  https://github.com/cloudflare/cloudflared/releases/download/2026.1.2/cloudflared-linux-amd64
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/106867604/31ade8d6-2fcd-4925-9218-5534d27a01dc?sp=r&sv=2018-11-09&sr=b&spr=https&se=2026-02-06T05%3A02%3A15Z&rscd=attachment%3B+filename%3Dcloudflared-linux-amd64&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2026-02-06T0

In [ ]:
!pkill streamlit
!streamlit run app.py & ./cloudflared tunnel --url http://localhost:8501

2026-02-06T04:20:42Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2026-02-06T04:20:42Z INF Requesting new quick Tunnel on trycloudflare.com...


2026-02-06T04:20:46Z INF +--------------------------------------------------------------------------------------------+
2026-02-06T04:20:46Z INF |  Your quick Tunnel has been created! Visit it at (it may take some time to be reachable):  |
2026-02-06T04:20:46Z INF |  https://clinics-added-region-britannica.trycloudfla

In [ ]:
!zip -r my_project.zip . -x "sample_data/*" ".config/*"

  adding: amanpay_transactions.csv (deflated 71%)
  adding: amanpay_model.keras (deflated 86%)
  adding: cloudflared (deflated 51%)
  adding: key.json (deflated 30%)
  adding: app.py (deflated 54%)
  adding: scaler.pkl (deflated 28%)
  adding: amanpay_cleaned.csv (deflated 71%)
